In [1]:
# build_stores_analysis.py
import os, glob, zipfile, nbformat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from datetime import datetime

In [2]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10,6)

In [3]:
ROOT = os.getcwd()
CSV_NAME_GLOB = "Stores*.csv"  # matches Stores.csv or Stores (1).csv etc.
PDF_NAME = "Supermarket Store Branches.pdf"
ASSETS_DIR = os.path.join(ROOT, "stores_assets")
os.makedirs(ASSETS_DIR, exist_ok=True)

In [8]:

# ---------- Helper: fuzzy column mapping ----------
def find_best_column(df_cols, target_variants):
    """
    df_cols : iterable of DataFrame columns (strings)
    target_variants : list of possible target names (strings)
    returns best match or None
    """
    cols_lower = [c.lower().strip() for c in df_cols]
    for tv in target_variants:
        tv_l = tv.lower()
        if tv_l in cols_lower:
            idx = cols_lower.index(tv_l)
            return list(df_cols)[idx]
    # try contains matching (partial)
    for tv in target_variants:
        tv_l = tv.lower().replace("_"," ").strip()
        for c in df_cols:
            if tv_l in c.lower().replace("_"," "):
                return c
    # fallback: try token matching
    for c in df_cols:
        for tv in target_variants:
            if all(tok in c.lower() for tok in tv.lower().split()):
                return c
    return None


In [9]:
# ---------- Find CSV ----------
csv_candidates = glob.glob(os.path.join(ROOT, CSV_NAME_GLOB))
if not csv_candidates:
    raise FileNotFoundError(f"No file matching '{CSV_NAME_GLOB}' found in {ROOT}. Please upload Stores.csv.")
csv_path = csv_candidates[0]
print("Using dataset:", csv_path)


Using dataset: C:\Users\Radejoro\Supermarket Store\Stores.csv


In [10]:

# ---------- Load CSV with fallbacks ----------
encodings = ['utf-8','latin1','cp1252','utf-16']
df = None
for e in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=e)
        print("Loaded CSV with encoding:", e)
        break
    except Exception as ex:
        print("Failed with encoding", e, ":", ex)
if df is None:
    raise ValueError("Failed to read CSV with tried encodings.")

# print detected columns
print("Detected columns:", list(df.columns))


Loaded CSV with encoding: utf-8
Detected columns: ['Store ID ', 'Store_Area', 'Items_Available', 'Daily_Customer_Count', 'Store_Sales']


In [11]:
# ---------- Map expected columns robustly ----------
expected = {
    "store_id": ["Store ID", "store id", "id", "storeid", "store"],
    "store_area": ["Store_Area", "Store Area", "store area", "area", "store_area"],
    "items_available": ["Items_Available", "Items Available", "items available", "items_available", "items"],
    "daily_customers": ["DailyCustomerCount", "Daily Customer Count", "daily customer", "dailycustomers", "customers", "daily_customer_count"],
    "store_sales": ["Store_Sales", "Store Sales", "sales", "store_sales", "store sales (usd)"]
}

mapped = {}
for key, variants in expected.items():
    col = find_best_column(df.columns, variants)
    mapped[key] = col
    print(f"Mapping {key} -> {col}")

# Validate mapping
missing = [k for k,v in mapped.items() if v is None]
if missing:
    print("ERROR: Could not automatically detect these required columns:", missing)
    print("Detected columns were:", list(df.columns))
    print("Please rename the CSV columns or update the mapping in the script.")
    raise SystemExit(1)

# normalize names in a working DataFrame
work = df.rename(columns={mapped['store_id']:'Store ID',
                          mapped['store_area']:'Store_Area',
                          mapped['items_available']:'Items_Available',
                          mapped['daily_customers']:'DailyCustomerCount',
                          mapped['store_sales']:'Store_Sales'}).copy()

# Convert numeric columns (coerce errors)
for c in ['Store_Area','Items_Available','DailyCustomerCount','Store_Sales']:
    work[c] = pd.to_numeric(work[c], errors='coerce')

# Basic cleaning
print("Rows before dropping NA:", len(work))
work = work.dropna(subset=['Store_Sales'])
print("Rows after dropping NA Store_Sales:", len(work))

# Create KPI columns
work['Sales_per_Customer'] = work['Store_Sales'] / work['DailyCustomerCount'].replace(0, np.nan)
work['Sales_per_Area'] = work['Store_Sales'] / work['Store_Area'].replace(0, np.nan)


Mapping store_id -> Store ID 
Mapping store_area -> Store_Area
Mapping items_available -> Items_Available
Mapping daily_customers -> Daily_Customer_Count
Mapping store_sales -> Store_Sales
Rows before dropping NA: 896
Rows after dropping NA Store_Sales: 896


In [15]:
# ---------- Plots (saved to assets) ----------
palette = ["#E76F51", "#F4A261", "#E63946", "#F5F5F5", "#9E2A2B"]

# 1: Sales distribution
plt.figure()
sns.histplot(work['Store_Sales'].dropna(), bins=20, kde=True, color=palette[0])
plt.title("Distribution of Store Sales")
plt.xlabel("Store Sales (USD)")
plt.tight_layout()
p1 = os.path.join(ASSETS_DIR, "sales_distribution.png")
plt.savefig(p1, dpi=200); plt.close()

In [16]:
# 2: Top 10 stores by sales
top10 = work.sort_values('Store_Sales', ascending=False).head(10)
plt.figure()
plt.bar(top10['Store ID'].astype(str), top10['Store_Sales'], color=palette[1])
plt.title("Top 10 Stores by Sales")
plt.ylabel("Sales (USD)")
plt.xticks(rotation=45)
plt.tight_layout()
p2 = os.path.join(ASSETS_DIR, "top10_stores.png")
plt.savefig(p2, dpi=200); plt.close()


In [17]:
# 3: Store_Area vs Sales (regression)
plt.figure()
sns.regplot(x='Store_Area', y='Store_Sales', data=work, scatter_kws={'s':50}, line_kws={'color':palette[2]})
plt.title("Store Area vs Store Sales")
plt.tight_layout()
p3 = os.path.join(ASSETS_DIR, "area_vs_sales.png")
plt.savefig(p3, dpi=200); plt.close()

In [18]:
# 4: DailyCustomerCount vs Sales
plt.figure()
sns.regplot(x='DailyCustomerCount', y='Store_Sales', data=work, scatter_kws={'s':50}, line_kws={'color':palette[3]})
plt.title("Daily Customer Count vs Store Sales")
plt.tight_layout()
p4 = os.path.join(ASSETS_DIR, "customers_vs_sales.png")
plt.savefig(p4, dpi=200); plt.close()


In [19]:
# 5: Pairplot numeric
nums = ['Store_Area','Items_Available','DailyCustomerCount','Store_Sales']
available_nums = [c for c in nums if c in work.columns]
if len(available_nums) >= 2:
    pairplot = sns.pairplot(work[available_nums], diag_kind='kde', corner=True, plot_kws={'s':30,'alpha':0.6})
    p5 = os.path.join(ASSETS_DIR, "pairplot_numeric.png")
    pairplot.savefig(p5, dpi=200)
    plt.close()
else:
    p5 = None


In [20]:
# 6: Correlation heatmap
plt.figure(figsize=(6,5))
corr = work[available_nums].corr()
sns.heatmap(corr, annot=True, cmap='OrRd', fmt=".2f")
plt.title("Correlation Matrix (Numeric Features)")
plt.tight_layout()
p6 = os.path.join(ASSETS_DIR, "correlation_matrix.png")
plt.savefig(p6, dpi=200); plt.close()

print("Plots saved in:", ASSETS_DIR)

Plots saved in: C:\Users\Radejoro\Supermarket Store\stores_assets


In [21]:
# ---------- Notebook creation (lightweight, references assets) ----------
nb = nbformat.v4.new_notebook()
cells = []
cells.append(nbformat.v4.new_markdown_cell("# Supermarket Store Branches — Sales Analysis (PPDAC)\n\nThis notebook was auto-generated. See plots in `stores_assets/`."))
cells.append(nbformat.v4.new_code_cell("import pandas as pd\npd.options.display.max_columns=50\nprint('Run this notebook in the same folder as Stores.csv')"))
cells.append(nbformat.v4.new_markdown_cell("## Visualizations (saved in stores_assets/)"))
img_display_code = "from IPython.display import Image, display\nimport os\nassets='stores_assets'\nfor f in ['sales_distribution.png','top10_stores.png','area_vs_sales.png','customers_vs_sales.png','pairplot_numeric.png','correlation_matrix.png']:\n    p=os.path.join(assets,f)\n    if os.path.exists(p): display(Image(p))\n    else: print('Missing',p)"
cells.append(nbformat.v4.new_code_cell(img_display_code))
nb['cells'] = cells
notebook_path = os.path.join(ROOT, "Supermarket_Stores_Sales_Analysis_PPDAC.ipynb")
with open(notebook_path, 'w', encoding='utf-8') as fh:
    nbformat.write(nb, fh)
print("Notebook written to", notebook_path)

Notebook written to C:\Users\Radejoro\Supermarket Store\Supermarket_Stores_Sales_Analysis_PPDAC.ipynb


In [22]:
# ---------- README ----------
readme_text = f"""# Supermarket-Store-Branches-Sales-Analysis-PPDAC

Author: Adejoro Raymond Olaotan

This repository contains an exploratory PPDAC analysis of store branch sales (Stores.csv).

**Files included**
- Supermarket_Stores_Sales_Analysis_PPDAC.ipynb
- Stores.csv
- Supermarket Store Branches.pdf (context)
- stores_assets/ (plots)
- Supermarket-Store-Branches-Analysis-PPDAC.zip

**Contact:** your-email@example.com

Reference: Supermarket Store Branches PDF. (See included PDF in repository)
"""
with open(os.path.join(ROOT, "README_stores.md"), 'w', encoding='utf-8') as f:
    f.write(readme_text)
print("README_stores.md written")

README_stores.md written
